# Day 5 - Part 2

## Empirical Distributions

# Distributions

## Probability Distribution
* Random quantity with various possible values
* “Probability distribution”:
    - All the possible values of the quantity
    - The probability of each of those values

## Empirical Distribution

* Based on observations
* Observations can be from repetitions of an experiment
* “Empirical Distribution”
    - All observed values
    - The proportion of counts of each value

### Example: Dice
* simulate a roll as a sample from a table

In [1]:
die =  (
    Table()
    .with_column('face', np.arange(1, 7, 1))
)
die

NameError: name 'Table' is not defined

In [2]:
# roll a single die
die.sample(1)

NameError: name 'die' is not defined

### The probability distribution is uniform

In [ ]:
bins =  np.arange(0.5, 6.6, 1)
die.hist('face', bins=bins)

### Roll the die and plot the empirical distribution
* Try it for 10, 100, 1000, etc
* What does it converge to?

In [ ]:
die.sample(10)

In [ ]:
die.sample(10).hist('face', bins=bins)

# Large Random Samples

## Law of Averages/Law of Large Numbers

* If a chance experiment is repeated 
    - many times,
    - independently,
    - under the same conditions,
    
then the proportion of times that an event occurs gets closer to the theoretical probability of the event.


*Example:* As you roll a die repeatedly, the proportion of times you roll a 5 gets closer to 1/6.

## Convergence of Empirical Histogram

If the sample size is large, then the empirical distribution of a uniform random sample resembles the distribution of the population, with high probability.

## Application: Estimating Probabilities through Simulation

* The law of averages justifies why we can estimate probabilities through simulation.
* If we simulate many times, the empirical proportion of times that an event occurs is likely to be very close to the true probability of the event.

### Estimating probability: rolling a die $N$ times

### Discussion Question

If you roll a die 4 times. What's P(at least one 6)?

|Option|Answer|
|---|---|
|A| $5/6$|
|B| $1-5/6$|
|C| $1-(5/6)^4$|
|D| $1-(1/6)^4$|
|E| None of the above|


### Answer for 4 rolls
$$ P(\text{at least one 6}) = 1 - P(\text{no 6}) = 1 - \left(\dfrac{5}{6}\right)^4$$

### Answer for N rolls
$$ P(\text{at least one 6}) = 1 - P(\text{no 6}) = 1 - \left(\dfrac{5}{6}\right)^N$$

### Plot the true probability for each N

In [ ]:
#:
rolls = np.arange(1, 51)
at_least_one = Table().with_columns('roll', rolls, 'Chance of getting at least one 6', 1-(5/6)**rolls)
at_least_one.scatter('roll')

### Estimate the probability for N=20
* What is the chance of getting at least one 6 in 20 rolls?

In [ ]:
faces = np.arange(1, 7)
outcomes = np.random.choice(faces, 20) # pick random number from faces, 20 times
outcomes

In [ ]:
# number of sixes


In [ ]:
rolled6 = 0
trials = 100000
for i in np.arange(trials):
    outcomes = np.random.choice(faces, 20)
    if np.count_nonzero(outcomes == 6) >=1:
        rolled6 = rolled6 + 1
        
#estimate the probability
rolled6/trials

### Estimate the probability for N=20
* wrap the experiment in a function
* run the experiment many times

In [ ]:
def roll_20(trials):
    rolled6 = 0
    for i in np.arange(trials):
        outcomes = np.random.choice(faces, 20)
        if np.count_nonzero(outcomes == 6) >=1:
            rolled6 = rolled6 + 1

    return rolled6/trials

roll_20(1000)

In [ ]:
estimates = make_array()
for i in np.arange(500):
    estimates = np.append(roll_20(1000), estimates)
    
probs = Table().with_column('estimates', estimates)

In [ ]:
probs.hist()
true_prob = 1 - (5/6)**20
plt.axvline(x=true_prob, c='r');

## Sampling from a Population

Knowing that large random samples resemble the population is extremly practical. This allows us to sample from a population instead of conducting a census.

### Example: distribution of flight delays
* All united flights leaving SFO between 6/1/15 and 8/9/15.
* The underlying distribution is not known.
* All we have is the observed data!

In [ ]:
#:
united = Table().read_table('united_summer2015.csv')
united

### Distribution of flight delays in the full population


In [ ]:
united.num_rows

In [ ]:
bins = np.arange(-20, 300, 10)
united.hist('Delay', bins=bins, unit='minute')

In [ ]:
# try larger N's
N = 100
united.sample(N).hist('Delay', bins=bins, unit='minute')

### Estimating the mean
* If the distribution of a sample looks like the distribution of the population, does the mean of the sample look like the mean of the population?
* Calculate the mean of all delays (population)
* Compare to the mean of uniform random samples.

In [ ]:
# calculate the mean of population
united_mean = united.column('Delay').mean()

In [ ]:
for n in np.arange(100, 10000, 200):
    m = united.sample(n).column('Delay').mean()
    print('number of flights: ', n, '    mean of sample: ', m)

### Distribution of means from uniform samples with replacement
* Nice curve around the mean.
* Does the histogram skew one direction?

In [ ]:
n_experiments = 10000
means = make_array()
for n in np.arange(n_experiments):
    m = united.sample(100).column('Delay').mean()
    means = np.append(m, means)

Table().with_columns('mean', means).hist(bins=np.arange(0,40))
plt.axvline(x=united_mean, c='r');

### Distribution of means from uniform samples without replacement


In [ ]:
n_experiments = 10000
means = make_array()
for n in np.arange(n_experiments):
    m = united.sample(100, with_replacement=False).column('Delay').mean()
    means = np.append(m, means)

Table().with_columns('mean', means).hist(bins=np.arange(0,40))
plt.axvline(x=united_mean, c='r');

### Distribution of means from uniform samples of flights from Denver

* How you sample matters.
* This sample is a probability sample. 
     - Each flight from Denver is equally likely to be in the sample.
     - Each flight from somewhere else is not in the sample.
* Estimation of the mean is highly biased!

In [ ]:
n_experiments = 10000
means = make_array()

den = united.where('Destination', 'DEN')
for n in np.arange(n_experiments):
    m = den.sample(100).column('Delay').mean()
    means = np.append(m, means)

Table().with_columns('mean', means).hist(bins=np.arange(0,40))
plt.axvline(x=united_mean, c='r');

### Distribution of means from evenly-spaced random samples
* This sample is a probability sample.
* Why does the histogram look this way?

In [ ]:
n_experiments = 10000
means = make_array()
for n in np.arange(n_experiments):
    start = np.random.choice(np.arange(20))
    m = united.take(np.arange(start, united.num_rows, 50)).column('Delay').mean()
    means = np.append(m, means)

Table().with_columns('mean', means).hist(bins=np.arange(0,40))
plt.axvline(x=united_mean, c='r');

### Distribution of means from deterministic samples
* Low variation and very high bias leads to confidence. 
* But we are way off because the sample was not taken uniformly at random.
* How you sample matters!

In [ ]:
#:
n_experiments = 10000
means = make_array()
for n in np.arange(n_experiments):
    m = united.take(np.arange(100)).column('Delay').mean()
    means = np.append(m, means)

Table().with_columns('mean', means).hist(bins=np.arange(0,40))
plt.axvline(x=united_mean, c='r');